In [ ]:
import imp
import torch
import torchvision
import torchvision.transforms as transforms

from context import rf_pool

In [ ]:
from rf_pool import modules, models
from rf_pool.utils import functions, datasets

In [ ]:
imp.reload(datasets)
imp.reload(functions)
imp.reload(modules)
imp.reload(models)

In [ ]:
# get MNIST training data
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='../rf_pool/data', train=True, download=True, transform=transform)

In [ ]:
# create trainloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)

In [ ]:
# initialize model
model = rf_pool.models.FeedForwardNetwork()

In [ ]:
# rf pooling layer
img_shape = torch.as_tensor((24,24))
mu, sigma = rf_pool.utils.lattice.init_uniform_lattice(img_shape//2, 4, 7, sigma_init=2.)
rf_layer = rf_pool.layers.RF_Pool(mu=mu, sigma=sigma, img_shape=img_shape, 
                                  lattice_fn=rf_pool.utils.lattice.mask_kernel_lattice,
                                  pool_type='max', kernel_size=2)
rf_layer.show_lattice()
print(mu.shape)
n_kernels = mu.shape[0]

In [ ]:
# append layers of model
model.append('0', rf_pool.modules.FeedForward(hidden=torch.nn.Conv2d(1,16,5), activation=torch.nn.ReLU(), 
                                           pool=rf_layer))
model.append('1', rf_pool.modules.FeedForward(hidden=torch.nn.Conv2d(16,32,5), activation=torch.nn.ReLU(),
                                           pool=torch.nn.MaxPool2d(2)))
model.append('2', rf_pool.modules.FeedForward(hidden=torch.nn.Conv2d(32,64,4)))
model.append('3', rf_pool.modules.FeedForward(input_shape=(-1, 64), hidden=torch.nn.Linear(64, 10)))

In [ ]:
# set loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# train model, monitor weightse
loss_history = model.train(1, trainloader, loss_fn, optim, monitor=100,
                           show_weights={'layer_id': '0', 'cmap': 'gray'})

In [ ]:
# optimize texture from first and second layer output
data = iter(trainloader).next()[0]
seed = torch.rand_like(data, requires_grad=True)
loss_fn = torch.nn.MSELoss()
optim = torch.optim.SGD([{'params': seed}], lr=0.5, momentum=0.9)
seed = model.optimize_texture(10000, data, seed, loss_fn, optim, ['0','1'], show_texture=[data, seed])